In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import cython
import tweepy
import json
import requests
import inspect
import re
from tqdm import tqdm
from time import time, sleep

In [2]:
# data = pd.read_csv("politicians.csv")
# print(data)

# usernames = data["Username"].tolist()
# ids = data["ids"].tolist()



In [3]:
with open("dev_keys_json.json", "r") as f:
    keys = json.load(f)
client = tweepy.Client(bearer_token=keys["Bearer_token"],
                       access_token = keys["API_key"],
                       access_token_secret = keys["API_secret_key"],
                       wait_on_rate_limit=True)


# empty_list_list = []
# for i in range(len(ids)):
#     empty_list_list.append({"ids":[], "usernames":[]})

# data["Following"] = empty_list_list

In [4]:
from ast import literal_eval

data = pd.read_csv("politicians_connections_two.csv")
print(data)
data["Following"] = data["Following"].apply(literal_eval)

print(data)

usernames = data["Username"].tolist()
ids = data["ids"].tolist()

          Party                      Name  Branch        Username  \
0    Republican             Pete Sessions   House    PeteSessions   
1    Democratic            Debbie Dingell   House   RepDebDingell   
2    Democratic           Pramila Jayapal   House      RepJayapal   
3    Republican               Rob Wittman   House      RobWittman   
4    Democratic           Chellie Pingree   House  chelliepingree   
..          ...                       ...     ...             ...   
521  Republican               Thom Tillis  Senate   SenThomTillis   
522  Republican         Diana Harshbarger   House  RepHarshbarger   
523  Republican          Tommy Tuberville  Senate   SenTuberville   
524  Democratic  Debbie Wasserman Schultz   House    RepDWStweets   
525  Republican              Lisa McClain   House  RepLisaMcClain   

                     ids                                          Following  
0               24735461  {'ids': [1396151678, 815966620300480514, 12912...  
1             2

In [5]:
def get_followings(client, id):
    followings = []

    for follow in tweepy.Paginator(client.get_users_following, id = id,
                                  max_results=1000).flatten(limit = 99999):
        followings.append(follow)
    
    return followings


pbar = tqdm(ids)


#i = 0
for i, id in enumerate(pbar):
    pbar.set_description(f"{usernames[i]}")
    
    #dict_form = ast.literal_eval(data.iloc[i, 5])
    #print(repr(dict_form))
    #data.iloc[i, 5] = dict_form
    
    #print(len(data["Following"].tolist()[i]["ids"]))
    if data["Following"].tolist()[i]["ids"] != []:
        #print(i)
        #i += 1
        continue
    
    try:
        followings = get_followings(client, id)
    except TweepError as tweeperror:
        pbar.set_description(f'Tweep error: {tweeperror}, sleeping for 60s')
        sleep(60)
        followings = get_followings(client, id)
    except HTTPException as tweeperror:
        pbar.set_description(f'Tweep error: {tweeperror}, sleeping for 60s')
        sleep(60)
        followings = get_followings(client, id)
        

    # for follow in tweepy.Paginator(client.get_users_following, id = id,
    #                               max_results=1000).flatten(limit = 99999):
    #     followings.append(follow)
    
    these_follow_ids = []
    these_follow_names = []
    
    for f in followings:
        #print(f)
        f_id = f["id"]
        f_username = f["username"]
        
        these_follow_ids.append(f_id)
        these_follow_names.append(f_username)
    #print(these_follow_ids)
    #print(these_follow_names)
    
    data.iloc[i, 5]["ids"] = these_follow_ids
    data.iloc[i, 5]["usernames"] = these_follow_names
    
    data.to_csv("politicians_connections_two.csv", index = False)
    
    i += 1

SenSchumer:   0%|                                             | 0/526 [00:00<?, ?it/s]Rate limit exceeded. Sleeping for 146 seconds.
Rate limit exceeded. Sleeping for 890 seconds.
MarkWarner:  46%|██████████████▍                | 244/526 [4:17:53<7:45:05, 98.95s/it]Rate limit exceeded. Sleeping for 889 seconds.
Rate limit exceeded. Sleeping for 889 seconds.
DarrellIssa:  57%|█████████████████▏            | 302/526 [6:48:08<3:17:58, 53.03s/it]Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 890 seconds.
RepJudyChu:  93%|██████████████████████████▉  | 488/526 [13:48:34<2:53:55, 274.62s/it]Rate limit exceeded. Sleeping for 891 seconds.
Rate limit exceeded. Sleeping for 891 seconds.
Rate limit exceeded. Sleeping for 890 seconds.
RepLisaMcClain: 100%|███████████████████████████| 526/526 [15:48:54<00:00, 108.24s/it]


In [6]:
data["Following_ids"] = following_ids
data["Following_usernames"] = following_usernames
# id_one = data["ids"][0]
# following = client.get_users_following(id = id_one)
# print(following[0])
# print(repr(following[0][0]["id"]))

NameError: name 'following_ids' is not defined